# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "2")
         .config("spark.executor.memory", "5g")
         .config("spark.executor.cores", "4")
         .config("spark.driver.memory", "5g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "10g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

21/09/24 16:45:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [3]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.filtrar_sentencias(preprocess=solo_portanto)

estructurales = SentenciasEstructurales(seleccion)

# Formar dataset de sentencias estructurales

In [4]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos()

# Cargar procesamiento de sentencias estructurales.

In [3]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)

In [17]:
estructurales.seleccion.sdf.toPandas()[estructurales.seleccion.sdf.toPandas()['plazo'] > 0]['txt'].iloc[[1]]

4    *060127740007CO* Exp: 06-012774-0007-CO Res. N...
Name: txt, dtype: object

In [26]:
print(estructurales.seleccion.sdf.filter("expediente == '170020150007CO'").select("txt").toPandas().loc[0,'txt'])

*170020150007CO* Exp: 17-002015-0007-CO Res. Nº 2017002763 SALA CONSTITUCIONAL DE LA CORTE SUPREMA DE JUSTICIA. San José, a las nueve horas cinco minutos del veintidos de febrero de dos mil diecisiete .
 Recurso de amparo que se tramita en expediente número 17-002015-0007-CO, interpuesto por [Nombre 001] , cédula de identidad [Valor 001], contra EL HOSPITAL DR. RAFAEL ÁNGEL CALDERÓN GUARDIA.
Resultando:
1.- Por escrito recibido en esta Sala a las 09:22 hrs. del 08 de febrero de 2017, la parte recurrente presenta recurso de amparo y expone que, es paciente del Servicio de Cirugía de Tórax y Cardiovascular del centro hospitalario recurrido, debido a un padecimiento cardíaco denominado "estenosis aortica severa e insuficiencia mitral moderada". Indica que la recomendación de su médico tratante fue recurrir a una cirugía inmediata, dada la urgencia de su caso; no obstante, se encuentra en una lista de espera desde el 2015. Según se le ha indicado, no se le puede intervenir, quirúrgicamente

In [11]:
from spacy.matcher import Matcher

txt = """ Se declara con lugar el recurso. Se ordena a Yaxinia Díaz Mendoza, en calidad de Directora de Recursos Humanos, a Rosa Adolio Cascante, en calidad de Viceministra Administrativa y a Walter Muñoz Caravaca en su calidad de Director de Infraestructura y Equipamiento, todos del Ministerio de Educación Pública o a quienes en su lugar ocupen los cargos, cada uno dentro del ámbito de sus competencias, que en el plazo máximo de OCHO DÍAS contados a partir de la notificación de esta sentencia realicen el trámite correspondiente para nombrar un Profesor de Enseñanza General Básica 1 en la Escuela La Katira. Deberán además en el plazo máximo de TRES MESES contados a partir de la notificación de esta sentencia iniciar con los procedimientos necesarios –procedimientos que incluyen tanto la aprobación de presupuesto como el proceso de contratación- para la construcción de la infraestructura requerida por la Escuela La Katira. En ambas órdenes se debe garantizar a los estudiantes perjudicados su continuidad en el proceso educativo. Se advierte a dichos funcionarios que de no acatar la orden dicha, incurrirán en el delito de desobediencia y, que de conformidad con el artículo 71 de la Ley de esta jurisdicción, se le impondrá prisión de tres meses a dos años, o de veinte a sesenta días multa, a quien recibiere una orden que deba cumplir o hacer cumplir, dictada en un recurso de amparo y no la cumpliere o no la hiciere cumplir, siempre que el delito no esté más gravemente penado. Se condena al Estado al pago de las costas, daños y perjuicios causados con los hechos que sirven de base a esta declaratoria, los que serán liquidados en ejecución de sentencia de lo contencioso administrativo. El Magistrado Salazar Alvarado pone nota. La Magistrada Esquivel Rodríguez consigna nota. Notifíquese la presente resolución a Yaxinia Díaz Mendoza, en calidad de Directora de Recursos Humanos, a Rosa Adolio Cascante, en calidad de Viceministra Administrativa y a Walter Muñoz Caravaca en su calidad de Director de Infraestructura y Equipamiento, todos del Ministerio de Educación Pública o a quienes en su lugar ocupen los cargos, EN FORMA PERSONAL. Deberán las autoridades recurridas informar a esta Sala sobre el avance en la construcción de las aulas en la Escuela La Katira. Comuníquese al Ministro de Educación Pública el contenido de esta sentencia.
"""
patron = [{"LOWER": "plazo"},
          {"TEXT": {"REGEX": "^(?!\.|[Hh][OoÓó][Rr][Aa]([Ss])?|[dD][iíIÍ][ÁAaa]([Ss])?|[Mm][EeÉé][Ss]([EeÉé][Ss])?|[ÁáAa][Ññ][OoÓó]([Ss])?)"}, "OP": "+"},
          {"TEXT": {"REGEX": "\.|[Hh][OoÓó][Rr][Aa]([Ss])?|[dD][iíIÍ][ÁAaa]([Ss])?|[Mm][EeÉé][Ss]([EeÉé][Ss])?|[ÁáAa][Ññ][OoÓó]([Ss])?"}},
          ]

patterns = [patron]

nlp = spark_get_spacy('es_core_news_lg')
doc = nlp(txt)
matcher = Matcher(nlp.vocab)
matcher.add("Patron 1 :", patterns, greedy="FIRST")

matches = matcher(doc)
print(matches)
plazos = []
for match_id, start, end in matches:
    includeText = False
    plazo = ""
    for token in doc[start:end]:
        if includeText:
            if token.pos_ == "PUNCT":
                break
            plazo += " " + token.text
        else:
            if token.pos_ == "NUM":
                plazo += token.text
                includeText = True
    if plazo != "":
        plazos.append(plazo)
                
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    print("SPAN")
    span =  doc[start:end]
    print(span.text)

print("PLAZOS")
print(plazos)

[(16778526997692001665, 76, 81), (16778526997692001665, 113, 118)]
SPAN
plazo máximo de OCHO DÍAS
SPAN
plazo máximo de TRES MESES
PLAZOS
['OCHO DÍAS', 'TRES MESES']


In [5]:
estructurales.sdf.toPandas()[estructurales.sdf.toPandas()['expediente'] == '040099160007CO']

,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,Ley_ents,EntidadPublica_ents,subtema_prob,Constitución_ents,Fecha_ents,anno,seguimiento,se_ordena,plan,plazo
46,8FAC1.html,\n*040099160007CO* Exp: 04-009916-0007-CO Res:...,TRAMITE,0.995852,040099160007CO,2006-03-08 14:48:00,Jinesta Lobo,Acción de Inconstitucionalidad,Sin lugar,True,...,"[Ley de la Jurisdicción Constitucional pues, L...","[Consejo Nacional de Cooperativas, Poder Ejecu...",NaN,"[182, 41 y 49, 18 y 129]","[28 de junio del 2004, 28 de junio del 2004, 7...",2006,0,0,0,5


In [42]:
from pyspark.sql.types import *
columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
resultado = estructurales.plazosDefinidos(columnas)
resultado.toPandas()

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,plan,plazo,se_ordena_PER,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,extension_sentencia,extension_por_lo_tanto,plazosDefinidos
0,72E91.html,*050139380007CO* Exp: 05-013938-0007-CO Res. N...,MUNICIPALIDAD,0.993193,050139380007CO,2006-03-28 18:21:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,0,0,"[Ronald Traña Calvo, Patricia Carmona Zeledón]",[Alcalde de la Municipalidad de Santa Ana],[Jefe del Departamento de Cobros de la Municip...,None,None,15786,1503,None
1,7A4A9.html,*060098620007CO* Exp: 06-009862-0007-CO Res. N...,PETICION,0.753669,060098620007CO,2006-09-13 17:17:00,Cruz Castro,Recurso de Amparo,Con lugar,False,...,0,1,"[Marco Vinicio Cordero Quesada, José Daniel Ar...",[Instituto de Desarrollo Agrario],"[Presidente Ejecutivo, Ley de la Jurisdicción ...",None,None,12572,1380,[1970-01-16 00:00:00]
2,7591A.html,*040023800007CO* Exp: 04-002380-0007-CO Res: 2...,AMBIENTE,0.999920,040023800007CO,2006-05-10 16:46:00,Jinesta Lobo,Recurso de Amparo,Con lugar parcial,False,...,0,0,"[Oscar Alvarado Bogantes, Fabio Molina Rojas, ...","[Urbanización El Portillo, Turrúcares de Alaju...","[Se ordena 1, Gerente General de la, Mutual Al...",None,None,58750,3136,None
3,75422.html,*060018790007CO* Exp: 06-001879-0007-CO Res: 0...,TRABAJO,1.000000,060018790007CO,2006-05-12 12:00:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,0,0,"[Soledad Chinchilla Arguedas, Subdirectora Gen...","[Ministerio de Educación Pública, Alba Iris Fe...",[No.2789780],None,None,14326,1391,None
4,7BC63.html,*060127740007CO* Exp: 06-012774-0007-CO Res. N...,PRONTA RESOLUCION,0.999759,060127740007CO,2006-11-07 17:32:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,0,2,"[Jeremías Vargas Chavarría, Franciso Morales H...",None,"[Director Nacional de Pensiones, Ministro de T...",None,None,10363,1912,"[1970-01-04 00:00:00, 1970-01-11 00:00:00]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,159e2.html,VOTO N 1570-92.- SALA CONSTITUCIONAL DE LA COR...,SERVICIOS PUBLICOS,0.999385,3368B91,1992-06-12 10:18:00,Baudrit- y,Acción de Inconstitucionalidad,None,False,...,0,1,None,None,None,None,None,1514,660,[1970-01-16 00:00:00]
941,153bc.html,VOTO N 1597-91.- SALA CONSTITUCIONAL DE LA COR...,PETICION,0.999375,1003B91,1991-08-16 09:48:00,Baudrit- y,Recurso de Amparo,Con lugar,False,...,0,1,None,[Ministerio],None,None,None,4579,1288,[1970-01-02 16:00:00]
942,14bb1.html,Voto 171-90 Fecha: 13-2-90 Hora: 14:45 Expedie...,SUJETO DE DERECHO PRIVADO,0.424996,469A89,NaT,Rodríguez Vega,Recurso de Amparo,Con lugar,False,...,0,0,None,[Sala],None,None,None,6002,808,None
943,14bd6.html,Voto 228-90 Fecha: 26-2-90 Hora: 15:00 Expedie...,TRAMITE,0.440360,4490,1990-02-26 15:00:00,Luis Fernando Solano Carrera AMPARO CONTRA SUJ...,Recurso de Amparo,Sin lugar,False,...,0,0,None,"[Ministerio Público, Garantía]","[Fondo de Ahorro, Vivienda, RECOPE]",None,None,6093,447,None


In [46]:
resultado.filter("plazosDefinidos is None").show()

ParseException: 
extraneous input 'None' expecting <EOF>(line 1, pos 19)

== SQL ==
plazosDefinidos is None
-------------------^^^


In [5]:
resultado.toPandas()[50:100]

,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,EntidadPublica_ents,subtema_prob,Constitución_ents,Fecha_ents,anno,seguimiento,se_ordena,plan,plazo,plazosDefinidos
50,E5580.html,*170015620007CO* Exp: 17-001562-0007-CO Res. N...,SALUD,1.000000,170015620007CO,2017-02-17 09:45:00,Salazar Murillo,Recurso de Amparo,Con lugar,False,...,"[Poder Judicial, Caja Costarricense de Seguro ...",0.999990,None,"[1 de febrero de 2017, 31 de enero de 2017, 12...",2017,0,1,0,1,"[30, TRES]"
51,E88CD.html,*170063170007CO* Exp: 17-006317-0007-CO Res. N...,PENSIONES ALIMENTARIAS,0.999998,170063170007CO,2017-05-09 09:15:00,Rueda Leal,Hábeas Corpus,Con lugar parcial,False,...,[Poder Judicial],0.528175,[38],"[26 de abril de 2017, 13 de marzo de 2017, 21 ...",2017,0,1,0,1,"[8, 54, 30, CINCO]"
52,E5BAB.html,*170015390007CO* Exp: 17-001539-0007-CO Res. N...,TRABAJO,1.000000,170015390007CO,2017-02-24 09:30:00,Hernandez Gutierrez,Recurso de Amparo,Con lugar parcial,True,...,"[Dirección General de Servicio Civil, Minister...",0.999998,"[192 y 193, 33 y 56]","[31 de enero del 2017, 3 de diciembre de 2012,...",2017,0,1,0,1,"[seis, seis, seis, 12, 30, 9:05, 18, 2016]"
53,EA4DC.html,*170071130007CO* Exp: 17-007113-0007-CO Res. N...,SALUD,1.000000,170071130007CO,2017-05-26 09:20:00,Rueda Leal,Recurso de Amparo,Con lugar,True,...,"[CAJA COSTARRICENSE DE SEGURO SOCIAL, Caja Cos...",0.999976,"[21, 73]","[9 de mayo de 2017, 15 de abril de 2015, 28 de...",2017,0,0,0,1,"[4, 3, 30, 3]"
54,E7DD6.html,*170045100007CO* Exp: 17-004510-0007-CO Res. N...,SEGURIDAD SOCIAL,1.000000,170045100007CO,2017-04-21 09:15:00,Campos Calvo,Recurso de Amparo,Con lugar,False,...,"[Caja Costarricense de Seguro Social, Caja Cos...",NaN,None,"[22 de marzo de 2017, 14 de febrero al 15, 22 ...",2017,0,1,0,1,"[30, 15]"
55,ED5CE.html,*170166870007CO* Exp: 17-016687-0007-CO Res. N...,SALUD,1.000000,170166870007CO,2017-11-08 09:15:00,Hernandez Gutierrez,Recurso de Amparo,Con lugar,False,...,"[Caja Costarricense de Seguro Social, Caja Cos...",0.906351,None,"[24 de octubre de 2017, 16 de mayo de 2017, 26...",2017,0,1,0,1,"[30, tres]"
56,EE74C.html,*170182370007CO* Exp: 17-018237-0007-CO Res. N...,SALUD,1.000000,170182370007CO,2017-12-15 09:20:00,Cruz Castro,Recurso de Amparo,Con lugar parcial,False,...,"[CAJA COSTARRICENSE DE SEGURO SOCIAL, Caja Cos...",1.000000,None,"[20 de noviembre de 2017, 7 de enero de 2016, ...",2017,0,1,0,1,"[30, TRES]"
57,E700B.html,*170045230007CO* Exp: 17-004523-0007-CO Res. N...,TRABAJO,1.000000,170045230007CO,2017-04-21 09:15:00,Hernandez Gutierrez,Recurso de Amparo,Con lugar,False,...,"[MINISTERIO DE EDUCACIÓN PÚBLICA, MINISTERIO D...",0.999992,None,"[22 de marzo de 2017, 16 de setiembre de 2014,...",2017,0,1,0,1,"[seis, 30, 9:05, 18, 2016]"
58,EC882.html,*170130290007CO* Exp: 17-013029-0007-CO Res. N...,SALUD,0.999996,170130290007CO,2017-10-13 09:15:00,Salazar Alvarado,Recurso de Amparo,Con lugar,False,...,"[CAJA COSTARRICENSE DE SEGURO SOCIAL, Caja Cos...",1.000000,None,"[22 de agosto de 2017, 24 de agosto de 2017, 1...",2017,0,1,0,1,"[30, tres]"
59,EEAA7.html,*170170110007CO* Exp: 17-017011-0007-CO Res. N...,PENITENCIARIO,1.000000,170170110007CO,2017-12-15 09:20:00,Rueda Leal,Recurso de Amparo,Con lugar parcial,False,...,"[Caja Costarricense de Seguro Social, Caja Cos...",1.000000,None,"[30 de octubre de 2017, 31 de octubre de 2017,...",2017,0,1,0,2,"[30, TRES, TRES]"


In [13]:
convertToText = Txt2Numbers()
number = convertToText.number("venti s")


Key error
venti s


In [9]:

print(pd.Timedelta('2 month'))

NameError: name 'pd' is not defined

In [17]:
t = "9:05"
print(t.isalpha())

False


In [28]:
import re
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

text = "meses"
regularExpresion = re.compile(r"[Mm][EeÉé][Ss]([EeÉé][Ss])?")
print(regularExpresion.search(text))
print(str(txt2Date("mes", 2)))

<re.Match object; span=(0, 5), match='meses'>


NameError: name 'txt2Date' is not defined

In [34]:
print(pd.Timestamp('2017-01-01T12'))

2017-01-01 12:00:00


In [35]:
print(pd.Timestamp(pd.to_datetime('1970-01-01') + pd.Timedelta(days=0, hours=15)))


1970-01-01 15:00:00


In [26]:
from datetime import datetime, timedelta
from dateutil.parser import parse
parse('2019-04-07') - timedelta(days=3)
# or, to get it as a string
(parse('1970-01-01') + timedelta(days=3.6)).strftime("%m/%d/%Y, %H:%M:%S")

'01/04/1970, 14:24:00'

ABC
0
0
ABC
0
1
ABC
0
2
ABC
0
3
ABC
0
4
ABC
0
5
ABC
1
0
ABC
1
1
ABC
1
2
ABC
1
3
ABC
1
4
ABC
1
5
ABC
2
0
ABC
2
1
ABC
2
2
ABC
2
3
ABC
2
4
ABC
2
5
ABC
3
0
ABC
3
1
ABC
3
2
ABC
3
3
ABC
3
4
ABC
3
5
ABC
4
0
ABC
4
1
ABC
4
2
ABC
4
3
ABC
4
4
ABC
4
5
ABC
5
0
ABC
5
1
ABC
5
2
ABC
5
3
ABC
5
4
ABC
5
5
ABC
6
0
ABC
6
1
ABC
6
2
ABC
6
3
ABC
6
4
ABC
6
5
ABC
7
0
ABC
7
1
ABC
7
2
ABC
7
3
ABC
7
4
ABC
7
5
ABC
8
0
ABC
8
1
ABC
8
2
ABC
8
3
ABC
8
4
ABC
8
5
ABC
9
0
ABC
9
1
ABC
9
2
ABC
9
3
ABC
9
4
ABC
9
5
